In [6]:
""" combine query predictions and output as a result json file """
import os
from utils import execute_all
path1 = "/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions_0.json"
path2 = "/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions_1.json"

# combine two json files
import json
with open(path1) as f:
    data1 = json.load(f)
with open(path2) as f:
    data2 = json.load(f)

data1.update(data2)

DB_PATH = os.path.join('data', 'mimic_iv', 'mimic_iv.sqlite')
predictions = execute_all(data1, db_path=DB_PATH, tag='pred')

with open("/home/yjahn/joint-learning/RESULTS/baselinev3-flan-t5-large/predictions.json", "w") as f:
    json.dump(predictions, f, indent=4)

### Hard Vote Ensemble

In [1]:
import json

PATHS = [
  "./RESULTS/baselinev3-flan-t5-large/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold0/predictions.json",
  "./RESULTS/baselinev3-flan-t5-large_fold1/predictions.json",
  "./RESULTS/baselinev4-flan-t5-text2sql-with-schema-v2/predictions.json",
]

In [2]:
import pandas as pd

df = pd.DataFrame(columns=["id"])
for idx, json_path in enumerate(PATHS):
  column_name = f"answer_{idx}"
  df[column_name] = None
  with open(json_path, "r") as f:
    data = json.load(f)
    if idx == 0: # initialize column
      df["id"] = list(data.keys())
    for key in data: # append data
      df.loc[df["id"] == key, column_name] = data[key]

df.head(10)

,id,answer_0,answer_1,answer_2,answer_3
0,d084d1f3c277e6827087bb44,null,null,null,null
1,a35a9346ab483d0db0f202ca,[['hemodialysis']],[['hemodialysis']],[['hemodialysis']],[['hemodialysis']]
2,144cd6f1acfad4416003c26c,"[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ...","[['0.9% sodium chloride'], ['acetaminophen'], ..."
3,b20d40188481222bfbb9b02f,[['2100-09-06 23:09:00']],[['2100-09-11 14:48:00']],[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']]
4,f5f185ff5f7901dc7c4dd711,[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']],[['pseudomonas aeruginosa']]
5,e6db613772003ec72d44ebe5,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...,[['central venous catheter placement with guid...
6,769ea1c5d6c42c47ac9a1735,[['0.0']],[['0.0']],[['0.0']],[['0.0']]
7,6730aa47b18b0105eb3dd8a2,"[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]","[['chest tube #1'], ['foley'], ['void']]"
8,9d0210cd4045e7f8e860ce69,null,error_pred,null,null
9,ffc47b7e01463f229eb09bce,[['0.0']],[['0.0']],[['0.0']],[['0.0']]


In [3]:
# find rows with different answers
diff_row_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if len(set(answers)) > 1:
    diff_row_idxs.append(idx)

df_diff = df.loc[diff_row_idxs]
print(f"Number of different rows: {len(df_diff)}")
df_diff.head(10)

Number of different rows: 359


,id,answer_0,answer_1,answer_2,answer_3
3,b20d40188481222bfbb9b02f,[['2100-09-06 23:09:00']],[['2100-09-11 14:48:00']],[['2100-09-06 23:09:00']],[['2100-09-06 23:09:00']]
8,9d0210cd4045e7f8e860ce69,null,error_pred,null,null
10,2afd67ca4b6eb596928b1024,error_pred,error_pred,error_pred,null
12,10f5ecdf9123785c95f2bff6,error_pred,"[['blood culture, routine'], ['c. difficile pc...",error_pred,[]
15,be6fb8ffe49a397f297313f4,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...",error_pred,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9..."
16,96fb52e3ec9186d401d3bfb1,null,error_pred,null,null
24,97abc537c029974692f9c1af,null,null,null,[['0.0']]
27,04987dfc639773e714441af1,null,[],"[['0.9% sodium chloride'], ['5% dextrose'], ['...","[['0.9% sodium chloride'], ['bag'], ['furosemi..."
31,4f51bf550b78551e0824cb6c,error_pred,error_pred,error_pred,null
33,44162aee19eabadbab9a0172,error_pred,error_pred,error_pred,null


In [4]:
# iterrate over rows and check error_pred
error_pred_idxs = []
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers:
    error_pred_idxs.append(idx)

df_error_pred = df.loc[error_pred_idxs]
print(f"Number of error_pred rows: {len(df_error_pred)}")
df_error_pred.head(10)

Number of error_pred rows: 199


,id,answer_0,answer_1,answer_2,answer_3
8,9d0210cd4045e7f8e860ce69,null,error_pred,null,null
10,2afd67ca4b6eb596928b1024,error_pred,error_pred,error_pred,null
12,10f5ecdf9123785c95f2bff6,error_pred,"[['blood culture, routine'], ['c. difficile pc...",error_pred,[]
15,be6fb8ffe49a397f297313f4,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...","[['dextrose 5%'], ['gastric meds'], ['nacl 0.9...",error_pred,"[['dextrose 5%'], ['gastric meds'], ['nacl 0.9..."
16,96fb52e3ec9186d401d3bfb1,null,error_pred,null,null
31,4f51bf550b78551e0824cb6c,error_pred,error_pred,error_pred,null
33,44162aee19eabadbab9a0172,error_pred,error_pred,error_pred,null
40,9255fc2165b8150eea428c60,[],error_pred,error_pred,null
42,c16736a1325749fa143299dc,null,error_pred,error_pred,null
44,9282129b6c0688c18cb0f2dd,error_pred,error_pred,error_pred,[['0.0']]


In [5]:
# write df as new json file
ensembled_dict = {}
for idx, row in df.iterrows():
  columns = list(row.keys())
  answer_columns = [col for col in columns if "answer" in col]
  answers = [row[col] for col in answer_columns]
  if "error_pred" in answers: # pass the difficult question
    ensembled_dict[row["id"]] = "null"
    continue
  if len(set(answers)) == 1:
    ensembled_dict[row["id"]] = answers[0]
  else:
    ensembled_dict[row["id"]] = "null"

In [8]:
import json

%cd ./RESULTS/ENSEMBLE
path = "./ensemble_v4_four.json"
zip_path = path.replace(".json", ".zip")
with open(path, "w") as f:
    json.dump(ensembled_dict, f, indent=4)

!zip $zip_path $path

[Errno 2] No such file or directory: './RESULTS/ENSEMBLE'
/home/yjahn/joint-learning/RESULTS/ENSEMBLE
  adding: ensemble_v4_four.json (deflated 74%)


/home/yjahn/miniconda3/envs/ehr/lib/python3.11/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
